# Кто выжил из пассажиров Титаника

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [8]:
# Читаем данные для обучения (есть целевая переменная Survived)
train_data = pd.read_csv('train.csv')

# Читаем данные для теста (нет целевой переменной Survived)
test_data = pd.read_csv('test.csv')

In [9]:
# Удаляем признаки, которые не несут полезной информации для предсказания выживания
# Из тренировочных данных
train_data_clean = train_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Из тестовых данных (сохраняем PassengerId для финального отчета)
test_passenger_ids = test_data['PassengerId']
test_data_clean = test_data.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

In [11]:
# =========== A) Процент потерянных данных при удалении пропусков ===========
print("=== Пропущенные значения в тренировочных данных ===")
train_missing = train_data_clean.isnull().sum()
train_total = len(train_data_clean)
print(train_missing[train_missing > 0])

# Считаем процент строк, которые будут потеряны при полном удалении
rows_with_any_missing = train_data_clean.isnull().any(axis=1).sum()
percent_lost = (rows_with_any_missing / train_total) * 100
print(f"\nСтрок с пропусками: {rows_with_any_missing} из {train_total}")
print(f"Процент потерянных данных при удалении пропусков: {percent_lost:.2f}%")

print("\n=== Пропущенные значения в тестовых данных ===")
test_missing = test_data_clean.isnull().sum()
test_total = len(test_data_clean)
print(test_missing[test_missing > 0])

# =========== B) Заполнение пропусков ===========
print("\n=== Заполнение пропусков ===")

# Решаем проблему с предупреждением о копировании
# Вместо inplace используем прямое присваивание

# Для Age
train_age_median = train_data_clean['Age'].median()
train_data_clean['Age'] = train_data_clean['Age'].fillna(train_age_median)
test_data_clean['Age'] = test_data_clean['Age'].fillna(train_age_median)

# Для Embarked
train_embarked_mode = train_data_clean['Embarked'].mode()[0]
train_data_clean['Embarked'] = train_data_clean['Embarked'].fillna(train_embarked_mode)
test_data_clean['Embarked'] = test_data_clean['Embarked'].fillna(train_embarked_mode)

# Для Fare в тестовых данных
test_fare_median = train_data_clean['Fare'].median()
test_data_clean['Fare'] = test_data_clean['Fare'].fillna(test_fare_median)

# Для Age_was_missing
train_data_clean['Age_was_missing'] = train_data['Age'].isnull().astype(int)
test_data_clean['Age_was_missing'] = test_data['Age'].isnull().astype(int)

print("=== Проверка после заполнения пропусков ===")
print("Тренировочные данные:")
print(train_data_clean.isnull().sum())
print("\nТестовые данные:")
print(test_data_clean.isnull().sum())

=== Пропущенные значения в тренировочных данных ===
Series([], dtype: int64)

Строк с пропусками: 0 из 891
Процент потерянных данных при удалении пропусков: 0.00%

=== Пропущенные значения в тестовых данных ===
Series([], dtype: int64)

=== Заполнение пропусков ===
=== Проверка после заполнения пропусков ===
Тренировочные данные:
Survived           0
Pclass             0
Sex                0
Age                0
SibSp              0
Parch              0
Fare               0
Embarked           0
Age_was_missing    0
dtype: int64

Тестовые данные:
Pclass             0
Sex                0
Age                0
SibSp              0
Parch              0
Fare               0
Embarked           0
Age_was_missing    0
dtype: int64


In [12]:
# Преобразуем категориальные переменные в числовые

# 1. Пол (Sex) - используем LabelEncoder (или просто mapping)
from sklearn.preprocessing import LabelEncoder

# Вариант 1: Простой mapping (male=0, female=1)
train_data_clean['Sex'] = train_data_clean['Sex'].map({'male': 0, 'female': 1})
test_data_clean['Sex'] = test_data_clean['Sex'].map({'male': 0, 'female': 1})

# 2. Порт отправления (Embarked) - используем pd.get_dummies
# Создаем one-hot encoding для Embarked
embarked_dummies_train = pd.get_dummies(train_data_clean['Embarked'], prefix='Embarked')
embarked_dummies_test = pd.get_dummies(test_data_clean['Embarked'], prefix='Embarked')

# Объединяем с основными данными
train_data_clean = pd.concat([train_data_clean, embarked_dummies_train], axis=1)
test_data_clean = pd.concat([test_data_clean, embarked_dummies_test], axis=1)

# Удаляем исходный столбец Embarked
train_data_clean = train_data_clean.drop('Embarked', axis=1)
test_data_clean = test_data_clean.drop('Embarked', axis=1)

# Убедимся, что в тестовых данных есть все те же колонки, что и в тренировочных
# (one-hot может создать разные колонки для разных наборов)
for col in train_data_clean.columns:
    if col not in test_data_clean.columns and col != 'Survived':
        test_data_clean[col] = 0

for col in test_data_clean.columns:
    if col not in train_data_clean.columns:
        train_data_clean[col] = 0

# Упорядочиваем колонки одинаково (кроме Survived в тренировочных)
common_columns = [col for col in test_data_clean.columns]
train_data_clean = train_data_clean[['Survived'] + common_columns]
test_data_clean = test_data_clean[common_columns]

print("Категориальные переменные преобразованы в числовые")
print(f"Колонки в тренировочных данных: {train_data_clean.shape[1]}")
print(f"Колонки в тестовых данных: {test_data_clean.shape[1]}")

Категориальные переменные преобразованы в числовые
Колонки в тренировочных данных: 11
Колонки в тестовых данных: 10


In [13]:
import numpy as np

# =========== Проверка на выбросы ===========

print("=== Проверка на выбросы ===")

# Для числовых признаков проверим выбросы с помощью межквартильного размаха (IQR)
numeric_columns = ['Age', 'SibSp', 'Parch', 'Fare']

# Создаем копии данных для анализа
train_for_analysis = train_data_clean.copy()
test_for_analysis = test_data_clean.copy()

print("\nАнализ выбросов в тренировочных данных:")

for col in numeric_columns:
    if col in train_for_analysis.columns:
        Q1 = train_for_analysis[col].quantile(0.25)
        Q3 = train_for_analysis[col].quantile(0.75)
        IQR = Q3 - Q1

        # Определяем границы для выбросов
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Находим выбросы
        outliers = train_for_analysis[(train_for_analysis[col] < lower_bound) |
                                      (train_for_analysis[col] > upper_bound)]

        print(f"\n{col}:")
        print(f"  Q1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}")
        print(f"  Границы: [{lower_bound:.2f}, {upper_bound:.2f}]")
        print(f"  Выбросов: {len(outliers)} ({len(outliers)/len(train_for_analysis)*100:.1f}%)")

        if len(outliers) > 0:
            print(f"  Min выброса: {outliers[col].min():.2f}" if outliers[col].min() < lower_bound else "",
                  f"Max выброса: {outliers[col].max():.2f}" if outliers[col].max() > upper_bound else "")

=== Проверка на выбросы ===

Анализ выбросов в тренировочных данных:

Age:
  Q1=22.00, Q3=35.00, IQR=13.00
  Границы: [2.50, 54.50]
  Выбросов: 66 (7.4%)
  Min выброса: 0.42 Max выброса: 80.00

SibSp:
  Q1=0.00, Q3=1.00, IQR=1.00
  Границы: [-1.50, 2.50]
  Выбросов: 46 (5.2%)
 Max выброса: 8.00

Parch:
  Q1=0.00, Q3=0.00, IQR=0.00
  Границы: [0.00, 0.00]
  Выбросов: 213 (23.9%)
 Max выброса: 6.00

Fare:
  Q1=7.91, Q3=31.00, IQR=23.09
  Границы: [-26.72, 65.63]
  Выбросов: 116 (13.0%)
 Max выброса: 512.33


In [14]:
# Подготовим данные для обучения
X_train = train_data_clean.drop('Survived', axis=1)
y_train = train_data_clean['Survived']
X_test = test_data_clean.copy()

print(f"Размер X_train: {X_train.shape}")
print(f"Размер y_train: {y_train.shape}")
print(f"Размер X_test: {X_test.shape}")

# Масштабируем признаки
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучаем модель логистической регрессии на преобразованных данных
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

print(f"\nМодель обучена")
print(f"Точность на тренировочных данных: {model.score(X_train_scaled, y_train):.2%}")

Размер X_train: (891, 10)
Размер y_train: (891,)
Размер X_test: (418, 10)

Модель обучена
Точность на тренировочных данных: 79.80%


In [15]:
# Пример использования модели для предсказания выживания нового пассажира

def predict_survival(passenger_data):
    """
    Предсказывает, выживет ли пассажир на Титанике

    passenger_data: словарь с данными пассажира
    """
    # Пример входных данных для одного пассажира
    example_passenger = {
        'Pclass': 3,        # Класс (1, 2, 3)
        'Sex': 'male',      # Пол ('male' или 'female')
        'Age': 30.0,        # Возраст
        'SibSp': 0,         # Братья/сестры на борту
        'Parch': 0,         # Родители/дети на борту
        'Fare': 7.25,       # Стоимость билета
        'Embarked': 'S'     # Порт отправления ('C', 'Q', 'S')
    }

    # Обновляем примерами данными (если переданы)
    if passenger_data:
        example_passenger.update(passenger_data)

    # Создаем DataFrame с одним пассажиром
    passenger_df = pd.DataFrame([example_passenger])

    # Удаляем те же признаки, что и при обучении
    passenger_clean = passenger_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], errors='ignore')

    # Заполняем пропуски (используем те же значения, что при обучении)
    passenger_clean['Age'] = passenger_clean['Age'].fillna(28.0)  # Медиана из тренировочных данных
    passenger_clean['Fare'] = passenger_clean['Fare'].fillna(14.45)  # Медиана из тренировочных данных
    passenger_clean['Embarked'] = passenger_clean['Embarked'].fillna('S')  # Мода из тренировочных данных

    # Добавляем признак пропуска возраста
    passenger_clean['Age_was_missing'] = 0  # Предполагаем, что возраст был заполнен

    # Преобразуем категориальные переменные
    passenger_clean['Sex'] = passenger_clean['Sex'].map({'male': 0, 'female': 1})

    # One-hot encoding для Embarked
    embarked_dummies = pd.get_dummies(passenger_clean['Embarked'], prefix='Embarked')
    passenger_clean = pd.concat([passenger_clean, embarked_dummies], axis=1)
    passenger_clean = passenger_clean.drop('Embarked', axis=1)

    # Добавляем недостающие колонки one-hot
    for col in X_train.columns:
        if col not in passenger_clean.columns:
            passenger_clean[col] = 0

    # Упорядочиваем колонки как в тренировочных данных
    passenger_clean = passenger_clean[X_train.columns]

    # Масштабируем
    passenger_scaled = scaler.transform(passenger_clean)

    # Делаем предсказание
    survived = model.predict(passenger_scaled)[0]
    probability = model.predict_proba(passenger_scaled)[0]

    # Формируем результат
    result = {
        'survived': bool(survived),
        'probability_survived': float(probability[1]),
        'probability_died': float(probability[0])
    }

    return result

# Пример использования 1: Мужчина, 3-й класс, 30 лет
print("=== Пример 1: Мужчина, 3-й класс ===")
male_3rd_class = {
    'Pclass': 3,
    'Sex': 'male',
    'Age': 30.0,
    'SibSp': 0,
    'Parch': 0,
    'Fare': 7.25,
    'Embarked': 'S'
}
result1 = predict_survival(male_3rd_class)
print(f"Предсказание: {'Выживет' if result1['survived'] else 'Не выживет'}")
print(f"Вероятность выживания: {result1['probability_survived']:.1%}")

# Пример использования 2: Женщина, 1-й класс, 25 лет
print("\n=== Пример 2: Женщина, 1-й класс ===")
female_1st_class = {
    'Pclass': 1,
    'Sex': 'female',
    'Age': 25.0,
    'SibSp': 0,
    'Parch': 0,
    'Fare': 100.0,
    'Embarked': 'C'
}
result2 = predict_survival(female_1st_class)
print(f"Предсказание: {'Выживет' if result2['survived'] else 'Не выживет'}")
print(f"Вероятность выживания: {result2['probability_survived']:.1%}")

# Пример использования 3: Женщина, 2-й класс, 8 лет (ребенок)
print("\n=== Пример 3: Девочка, 2-й класс ===")
female_child = {
    'Pclass': 2,
    'Sex': 'female',
    'Age': 8.0,
    'SibSp': 1,
    'Parch': 2,
    'Fare': 30.0,
    'Embarked': 'Q'
}
result3 = predict_survival(female_child)
print(f"Предсказание: {'Выживет' if result3['survived'] else 'Не выживет'}")
print(f"Вероятность выживания: {result3['probability_survived']:.1%}")

=== Пример 1: Мужчина, 3-й класс ===
Предсказание: Не выживет
Вероятность выживания: 9.5%

=== Пример 2: Женщина, 1-й класс ===
Предсказание: Выживет
Вероятность выживания: 96.8%

=== Пример 3: Девочка, 2-й класс ===
Предсказание: Выживет
Вероятность выживания: 91.7%
